In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/MLLabs/application_record.csv')
record = pd.read_csv('/content/drive/MyDrive/MLLabs/credit_record.csv')

In [ ]:
data.columns

In [ ]:
record.columns

In [ ]:
data.shape,record.shape

No of Rows in Application_Record:

In [ ]:
print('No of Rows in Application_Record: ',data.shape[0])
print('No of Columns in Application Record: ',data.shape[1])

In [ ]:
print('No of Rows in Credit Record: ',record.shape[0])
print('No of col in Credit Record: ',record.shape[1])

In [ ]:
data.dtypes

In [ ]:
record.dtypes

In [ ]:
data.head()

In [ ]:
record.head()

# Preprocessing

On Record Dataset

In [ ]:
record['STATUS'].unique()


  0 : 1 Month past due

  1 : 1-2 Month past due

  2 : 2-3 Month Overdue

  3 : 3-4 Month Overdue

  4 : 4-5 Month Overdue

  5 : Overdue or Bad Debts

  C : Paid Off that month

  X : No loan for the month


In [ ]:
begin_months = record.groupby(['ID'])['MONTHS_BALANCE'].agg(min)

In [ ]:
begin_month=pd.DataFrame(begin_months)
begin_month.rename(columns={'MONTHS_BALANCE':'Begin Months'},inplace=True)
new_data=pd.merge(data,begin_month,how="left",on="ID")

In [ ]:
new_data.head()

In [ ]:
# Here new target column is created annd according to status value target 'yes' is decided
record['target'] = None
record.loc[record['STATUS'].isin(['2', '3', '4', '5']), 'target'] = 'Yes'

In [ ]:
record.head()

In [ ]:
record['target'].unique()

In [ ]:
temp = record.groupby('ID').count()
print(temp['target'].unique())
temp['target'][temp['target'] > 0] = 'Yes' # Specify Default
temp['target'][temp['target'] == 0] = 'No' # Specify Non Default
temp = temp[['target']]  # Only store target column
new_data = pd.merge(new_data, temp, how='inner', on='ID')
new_data['target'] = new_data['target'].map({'Yes': 1, 'No': 0})
new_data # here it includes target column

In [ ]:
new_data['target'].unique()

## Feature Engineering

Days Birth Column

In [ ]:
new_data['Age'] = new_data['DAYS_BIRTH'].abs()//365
print(new_data['Age'])
new_data.drop(['DAYS_BIRTH'],axis=1,inplace=True)

Days Employed Column

In [ ]:
new_data['Job Experiance'] = new_data['DAYS_EMPLOYED'].abs()//365
print(new_data['Job Experiance'])
new_data.drop(['DAYS_EMPLOYED'],axis=1,inplace=True)

## Renaming Columns

In [ ]:
new_data.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Reality',
                         'CNT_CHILDREN':'Total Children','AMT_INCOME_TOTAL':'Income',
                         'NAME_EDUCATION_TYPE':'Education','NAME_FAMILY_STATUS':'Marital Status',
                        'NAME_HOUSING_TYPE':'Housing Type','FLAG_EMAIL':'Email','FLAG_MOBIL':'Mobile',
                         'NAME_INCOME_TYPE':'Income Type','FLAG_WORK_PHONE':'Work Phone',
                         'FLAG_PHONE':'Phone','CNT_FAM_MEMBERS':'Total Family Members',
                        'OCCUPATION_TYPE':'Occupation Type','target':'Target'
                        },inplace=True)
new_data.columns


## Separate Categorical And Numerical Columns

In [ ]:
num_col = new_data.select_dtypes(exclude=object).columns
cat_col = new_data.select_dtypes(include=object).columns
print('Numerical Column: \n',num_col)
print('\nCategorical Columns:\n',cat_col)

In [ ]:
num_col = num_col.drop('Target')
num_col = num_col.drop('ID')

## Missing Values

In [ ]:
new_data.isna().sum()

In [ ]:
# See Percent Missing Values in Occupation Type
miss_rows = new_data['Occupation Type'].isna().sum()
total_rows = new_data['Occupation Type'].shape[0]
print('Missing Rows: ',miss_rows)
print('Total Rows: ',total_rows)
percent_missing = (miss_rows/total_rows)*100
print('Missing Percentage: ',percent_missing)

In [ ]:
new_data['Occupation Type'].unique()

In [ ]:
new_data['Occupation Type'].fillna('Unknown',inplace=True)

In [ ]:
new_data.isna().sum()

In [ ]:
new_data['Target'][new_data['Occupation Type']=='Unknown'].value_counts()

In [ ]:
new_data

# Total 1 and 0

In [ ]:
# Calculate percentage of 1's and 0's in dataset
# 1: Default
# 0: Non Default
zeros = new_data['Target'].value_counts()[0]
ones = new_data['Target'].value_counts()[1]
print('No of Ones: ',ones)
print('No of Zeros: ',zeros)
print('Percentage of Ones: ',(ones/(ones+zeros))*100)
print('Percentage of Zeros: ',(zeros/(ones+zeros))*100)

## Drop Duplicates

In [ ]:
new_data.drop_duplicates(inplace=True)

In [ ]:
new_data.describe()

## Encoding

In [ ]:
cat_col

In [ ]:
for col in cat_col:
  print(col,': ',new_data[col].unique())

### Marital Status

In [ ]:
for status in new_data['Marital Status'].unique():
  print(status,':',new_data['Marital Status'][new_data['Marital Status']==status].value_counts()[0])

In [ ]:
new_data['Marital Status'][new_data['Marital Status']=='Civil marriage'] = 'Married'
new_data['Marital Status'][new_data['Marital Status']=='Single / not married'] = 'Single'
new_data['Marital Status'][new_data['Marital Status']=='Separated'] = 'Single'
new_data['Marital Status'][new_data['Marital Status']=='Widow'] = 'Single'

In [ ]:
for status in new_data['Marital Status'].unique():
  print(status,':',new_data['Marital Status'][new_data['Marital Status']==status].value_counts()[0])

In [ ]:
new_data.to_csv('Dataset.csv',index=False)

## Income type

In [ ]:
new_data['Income Type'].unique()

## One Hot Encoding

In [ ]:
nominal_col = ['Gender','Car','Reality','Marital Status']

In [ ]:
new_data_ohe = pd.get_dummies(new_data,columns=cat_col)

In [ ]:
new_data_ohe.head()

# Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
sc.fit(new_data_ohe.loc[:,num_col])

In [ ]:
new_data_ohe.loc[:,num_col] = sc.transform(new_data_ohe.loc[:,num_col])

In [ ]:
new_data_ohe.head()

In [ ]:
new_data.describe()

# Outlier Handling

In [ ]:
def outlier_treament_zscore(df , cont_columns):
    """
    This is a function for treatment of outliers
    In given data column by columns
    Here Z-score / Standard scaler technique is used
    Z_score = (x-mu)/sigma
    mu => mean of the column
    sigma => std_dev of a column
    Here we
    replace all outliers in every column one by one
    when value < -3 replace with -3 and value > 3 replace with 3
    """
    for col in cont_columns:
        df.loc[df[col] < -3 , col] = -3
        df.loc[df[col] > 3 , col] = 3
    return df

In [ ]:
new_data = outlier_treament_zscore(new_data_ohe,num_col)

In [ ]:
new_data.describe()

In [ ]:
new_data.dtypes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(30,18))
sns.heatmap(new_data.corr(),cmap='summer', annot=True, square=True,  )
plt.show()

# X and Y

In [ ]:
Y = new_data['Target']
X = new_data.drop(['Target','ID'],axis=1)

In [ ]:
X

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,
                                                    stratify=Y, test_size=0.3,
                                                    random_state = 100)

# Unsupervised Algorithm

# Isolation Forest

In [ ]:
fraud_trans = new_data['Target'][new_data['Target']==1]
norm_trans = new_data['Target'][new_data['Target']==0]

In [ ]:
contamin = float(len(fraud_trans)/float(len(norm_trans)))

from sklearn.ensemble import IsolationForest

model = IsolationForest(random_state=10)
model.fit(X_train)
y_pred = model.predict(X_train)

y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
train_score = accuracy_score(y_train,y_pred)

In [ ]:
train_score

# One Class SVM

In [ ]:
from sklearn.svm import OneClassSVM

# Create the One-class SVM object
clf = OneClassSVM(kernel='rbf',nu=0.01)

# Fit the data and tag the outliers
clf.fit(X_train)

# Get the predictions
y_pred = clf.predict(X_train)

# Reshape the predictions to a 1D array
y_pred = y_pred.reshape(-1,1)

y_pred[y_pred==-1] =1
y_pred[y_pred==1] = 0

In [ ]:
accuracy_score(y_train,y_pred)